In [1]:
from tools import Artist, EDA, SuematsuData, metapipeline
from tools import StratifiedGOAnalysis as SGOA
from tools import eda_longitudal_args as ela
from tools import sgoa_longitudal_args as sla

In [2]:
data = SuematsuData()
metapipeline([Artist(), EDA(data), EDA(data, args=ela), SGOA(data), SGOA(data, **sla)])